### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [ ]:
purchase_data.head()

In [ ]:
# Push the remade DataFrame to a new CSV file
#purchase_data.to_csv("Output/purchase_data_clean.csv",
                  #encoding="utf-8", index=False, header=True)

## Player Count

In [ ]:
playerCount = purchase_data["SN"].nunique()

# new df with player info
players_df = pd.DataFrame([{'Total Players': playerCount}])
#gets rid of number index and resets to Total Players 
players_df.set_index('Total Players', inplace = True)
players_df

* Display the total number of players


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [ ]:
# Calculate the number of unique items in the DataFrame
unique_items = len(purchase_data["Item Name"].unique())
print(unique_items)

In [ ]:
# Calculate the Average Purchase Price
appAVG = purchase_data["Price"].mean()
print(appAVG)

In [ ]:
# Calculate the Total Number of Purchases
app = purchase_data["Price"].count()
print(app)

In [ ]:
# Calculate the total revenue
totalREV = purchase_data["Price"].sum()
print(totalREV)

In [ ]:
summary_table = pd.DataFrame({"Total Unique Items": [unique_items],
                              "Average Purchase Price": [appAVG],
                              "Total Purchases": [app],
                              "Total Revenue": [totalREV]})
summary_table

summary_table.style.format({'Average Purchase Price': '${:.2f}', 'Total Revenue': '${:,.2f}'})

## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [ ]:
# quick check for complete data
purchase_data.count()

In [ ]:
#remove duplicate SNs
no_dup_players = purchase_data.drop_duplicates(['SN'], keep ='last')

# count of genders 
gender_counts = no_dup_players['Gender'].value_counts().reset_index()
# add column for % of players using player count from first table and gender_counts 
# column which is a count from line above
gender_counts['% of Players'] = gender_counts['Gender']/playerCount * 100
# rename the columns
gender_counts.rename(columns = {'index': 'Gender', 'Gender': '# of Players'}, inplace = True)
# sets index as Gender
gender_counts.set_index(['Gender'], inplace = True)
#formats table
gender_counts.style.format({"% of Players": "{:.1f}%"})


## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [ ]:
purchase_data["Total Price"] = purchase_data["Price"]
purchase_data["Count"] = 0
purchase_analysis = purchase_data.groupby(["Gender"]).agg({"Price":"mean","Total Price":"sum","Count":"size"})
purchase_analysis["Normalized Totals"] = purchase_analysis["Total Price"] / purchase_analysis["Count"]
purchase_analysis = purchase_analysis.rename(columns={"Count": "Purchase Count","Price" : "Average Purchase Price","Total Price":"Total Purchase Value"})

# Convert to Dataframe to display tabular format
purchase_analysis = pd.DataFrame(purchase_analysis)

# organize data
purchase_analysis = purchase_analysis[["Purchase Count","Average Purchase Price","Total Purchase Value","Normalized Totals"]]

# cleanup on aisle 3
purchase_analysis["Average Purchase Price"] = purchase_analysis["Average Purchase Price"].map('${:,.2f}'.format)
purchase_analysis["Total Purchase Value"] = purchase_analysis["Total Purchase Value"].map('${:,.2f}'.format)
purchase_analysis["Normalized Totals"] = purchase_analysis["Normalized Totals"].map('${:,.2f}'.format)

purchase_analysis


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [ ]:
age_buckets = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
pd.options.display.float_format = '{:,.2f}'.format
df_grpby = purchase_data.copy()
df_grpby.drop_duplicates(['SN', 'Gender'], inplace=True, keep='first')
df_grpby.head()
df_grpby['Age_bin'] = pd.cut(df_grpby['Age'], age_buckets, labels=group_names)
df_grpby.head()
df = pd.DataFrame()
df['% of Players']=(df_grpby.groupby(['Age_bin'])['Age'].count() * 100)/float(playerCount)
df['Total Count'] = df_grpby.groupby(['Age_bin'])['Age'].count()
del df.index.name
df['% of Players']=df['% of Players'].map("%{:,.2f}".format)

df

## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [ ]:
age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

purchase_analysis = purchase_data

# add column and bucket info
purchase_analysis["Age Buckets"] = pd.cut(purchase_analysis["Age"], age_bins, labels = group_names)

# add columns: Purchase Count, Average Purchase Price, Total Purchase Value, Normalized Totals and set to 0
purchase_analysis["Purchase Count"] = 0
purchase_analysis["Average Purchase Price"] = purchase_analysis["Price"]
purchase_analysis["Total Purchase Value"] = purchase_analysis["Price"]
purchase_analysis["Normalized Totals"] = purchase_analysis["Price"]

# group 
purchase_analysis = purchase_analysis.groupby(["Age Buckets"]).agg({"Purchase Count" : "size","Average Purchase Price": "mean","Total Purchase Value" : "sum","Normalized Totals" : "sum","Age Buckets": "min"})
purchase_analysis["Normalized Totals"] = purchase_analysis["Normalized Totals"] / purchase_analysis["Purchase Count"]


# cleanup
purchase_analysis["Average Purchase Price"] = purchase_analysis["Average Purchase Price"].map("${:,.2f}".format)
purchase_analysis["Total Purchase Value"] = purchase_analysis["Total Purchase Value"].map("${:,.2f}".format)
purchase_analysis["Normalized Totals"] = purchase_analysis["Normalized Totals"].map("${:,.2f}".format)

# sort
purchase_analysis = purchase_analysis.sort_values(by=["Age Buckets"], ascending=True)

# remove unnecessary column
del purchase_analysis["Age Buckets"]

purchase_analysis

## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [ ]:
# grouping...again
top_spenders = purchase_data
top_spenders ["Total Price"] =top_spenders["Price"]
top_spenders ["Count"] = 0
top_spenders =top_spenders.groupby(["SN"]).agg({"Price":"mean","Total Price":"sum","Count":"size"})

# Sort
top_spenders =top_spenders.sort_values(by=["Total Price"], ascending=False)

# Organize
top_spenders =top_spenders.rename(columns={"Count": "Purchase Count","Price" : "Average Purchase Price","Total Price":"Total Purchase Value"})

# return to DF
top_spenders = pd.DataFrame(top_spenders)

# Organize DF
top_spenders =top_spenders[["Purchase Count","Average Purchase Price","Total Purchase Value"]]

# cleanup on aisle 3
top_spenders ["Average Purchase Price"] =top_spenders["Average Purchase Price"].map('${:,.2f}'.format)
top_spenders ["Total Purchase Value"] =top_spenders["Total Purchase Value"].map('${:,.2f}'.format)
top_spenders.head(5)

## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [ ]:
# group
most_popular = purchase_data
most_popular["Total Price"] = most_popular["Price"]
most_popular["Count"] = 0
most_popular = most_popular.groupby(["Item ID"]).agg({"Item Name":"min","Price":"min","Total Price":"sum","Count":"size"})

# sort
most_popular = most_popular.sort_values(by=["Count"], ascending=False)

# organize
most_popular = most_popular.rename(columns={"Count": "Purchase Count","Price" : "Item Price","Total Price":"Total Purchase Value"})

# return to df
most_popular = pd.DataFrame(most_popular)

# organize df
most_popular = most_popular[["Item Name","Purchase Count","Item Price","Total Purchase Value"]]

# cleanup
most_popular["Item Price"] =most_popular["Item Price"].map('${:,.2f}'.format)
most_popular["Total Purchase Value"] =most_popular["Total Purchase Value"].map('${:,.2f}'.format)
most_popular.head(5)

## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [ ]:
# group
most_profitable = purchase_data
most_profitable["Total Price"] = most_profitable["Price"]
most_profitable["Count"] = 0
most_profitable = most_profitable.groupby(["Item ID"]).agg({"Item Name":"min","Price":"min","Total Price":"sum","Count":"size"})
 
# Sort
most_profitable = most_profitable.sort_values(by=["Total Price"], ascending=False)

# organize
most_profitable = most_profitable.rename(columns={"Count": "Purchase Count","Price" : "Item Price","Total Price":"Total Purchase Value"})

# back to df
most_profitable = pd.DataFrame(most_profitable)

# organize
most_profitable = most_profitable[["Item Name","Purchase Count","Item Price","Total Purchase Value"]]

# cleanup
most_profitable["Item Price"] = most_profitable["Item Price"].map('${:,.2f}'.format)
most_profitable["Total Purchase Value"] = most_profitable["Total Purchase Value"].map('${:,.2f}'.format)
most_profitable.head(5)